In [40]:
import pandas as pd
import numpy as np
import os
from joblib import dump, load
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from dotenv import load_dotenv

In [2]:
load_dotenv()
data_basepath = os.environ['DATA_BASEPATH']
fn = 'Bonität Auszug 14-10-2022 export.csv'

In [17]:
df = pd.read_csv(os.path.join(data_basepath, fn), sep=';')

In [18]:
y = df['Risiko']

In [19]:
df['Geschlecht']=df['Geschlecht'].replace({0:'weiblich', 1:'männlich'})

In [20]:
df_X = df.drop(['Index', 'Risiko'], axis=1)

In [21]:
df_X

,Alter,Geschlecht,Arbeitsstelle,Wohnsituation,Sparkonto,Lohnkonto,Kreditbetrag,Laufzeit,Verwendungszweck
0,67,weiblich,mittlere Qualifikation,Eigentum,NaN,gering,5076,6,Unterhaltungselektronik
1,22,männlich,mittlere Qualifikation,Eigentum,gering,mittel,25840,48,Unterhaltungselektronik
2,49,weiblich,niedrige Qualifikation - permanent,Eigentum,gering,NaN,9101,12,Ausbildung
3,45,weiblich,mittlere Qualifikation,kostenlos,gering,gering,34224,42,Wohnungsreinrichtung
4,53,weiblich,mittlere Qualifikation,kostenlos,gering,gering,21146,24,Fahrzeug
...,...,...,...,...,...,...,...,...,...
992,31,männlich,niedrige Qualifikation - permanent,Eigentum,gering,NaN,7538,12,Wohnungsreinrichtung
993,40,weiblich,hohe Qualifikation,Eigentum,gering,gering,16748,30,Fahrzeug
994,38,weiblich,mittlere Qualifikation,Eigentum,gering,NaN,3491,12,Unterhaltungselektronik
995,23,weiblich,mittlere Qualifikation,kostenlos,gering,gering,8011,45,Unterhaltungselektronik


In [22]:
cat_features = ['Geschlecht','Arbeitsstelle','Wohnsituation','Sparkonto','Lohnkonto','Verwendungszweck']
num_features = ['Alter', 'Kreditbetrag', 'Laufzeit']

In [23]:
df_X[cat_features]

,Geschlecht,Arbeitsstelle,Wohnsituation,Sparkonto,Lohnkonto,Verwendungszweck
0,weiblich,mittlere Qualifikation,Eigentum,NaN,gering,Unterhaltungselektronik
1,männlich,mittlere Qualifikation,Eigentum,gering,mittel,Unterhaltungselektronik
2,weiblich,niedrige Qualifikation - permanent,Eigentum,gering,NaN,Ausbildung
3,weiblich,mittlere Qualifikation,kostenlos,gering,gering,Wohnungsreinrichtung
4,weiblich,mittlere Qualifikation,kostenlos,gering,gering,Fahrzeug
...,...,...,...,...,...,...
992,männlich,niedrige Qualifikation - permanent,Eigentum,gering,NaN,Wohnungsreinrichtung
993,weiblich,hohe Qualifikation,Eigentum,gering,gering,Fahrzeug
994,weiblich,mittlere Qualifikation,Eigentum,gering,NaN,Unterhaltungselektronik
995,weiblich,mittlere Qualifikation,kostenlos,gering,gering,Unterhaltungselektronik


In [46]:
set_config(display="diagram")

In [27]:
cat_transformer = Pipeline(
    steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore',dtype='int')),
        ('imputer', SimpleImputer(strategy="constant", fill_value=-1))]
)

In [28]:
preprocessor = ColumnTransformer(transformers=[('cat', cat_transformer, cat_features)], remainder='passthrough', verbose_feature_names_out=False)

In [47]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(dtype='int',
                                                                handle_unknown='ignore')),
                                                 ('imputer',
                                                  SimpleImputer(fill_value=-1,
                                                                strategy='constant'))]),
                                 ['Geschlecht', 'Arbeitsstelle',
                                  'Wohnsituation', 'Sparkonto', 'Lohnkonto',
                                  'Verwendungszweck'])],
                  verbose_feature_names_out=False)

In [29]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor),
           ("classifier", LogisticRegression(max_iter=10000))
          ]
)

In [41]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(dtype='int',
                                                                                 handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(fill_value=-1,
                                                                                 strategy='constant'))]),
                                                  ['Geschlecht',
                                                   'Arbeitsstelle',
                                                   'Wohnsituation', 'Sparkonto',
                                                   'Lohnkonto',
                                                   'Verwendungszweck'])],
                                   verbose_feature_names_out=False)),
                ('classifier', LogisticRegression(max_iter=10000))])

In [30]:
X_train, X_test, y_train, y_test = train_test_split(df_X, y, test_size=0.2, random_state=42)

In [31]:
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.730


In [44]:
clf.named_steps['preprocessor'].transform(df_X)

array([[    0,     1,     0, ...,    67,  5076,     6],
       [    1,     0,     0, ...,    22, 25840,    48],
       [    0,     1,     0, ...,    49,  9101,    12],
       ...,
       [    0,     1,     0, ...,    38,  3491,    12],
       [    0,     1,     0, ...,    23,  8011,    45],
       [    0,     1,     0, ...,    27, 19869,    45]], dtype=int64)

In [45]:
clf.named_steps['preprocessor'].

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(dtype='int',
                                                                handle_unknown='ignore')),
                                                 ('imputer',
                                                  SimpleImputer(fill_value=-1,
                                                                strategy='constant'))]),
                                 ['Geschlecht', 'Arbeitsstelle',
                                  'Wohnsituation', 'Sparkonto', 'Lohnkonto',
                                  'Verwendungszweck'])],
                  verbose_feature_names_out=False)

In [52]:
df_X.columns

Index(['Alter', 'Geschlecht', 'Arbeitsstelle', 'Wohnsituation', 'Sparkonto',
       'Lohnkonto', 'Kreditbetrag', 'Laufzeit', 'Verwendungszweck'],
      dtype='object')

In [68]:
data=[{'Alter': 67, 'Geschlecht': 'weiblich', 'Arbeitsstelle': 'mittlere Qualifikation', 'Wohnsituation': 'Eigentum', 'Sparkonto': pd.NA, 'Lohnkonto': 'gering', 'Kreditbetrag': 5076, 'Laufzeit': 6, 'Verwendungszweck': 'Unterhaltungselektronik'}]

In [69]:
clf.named_steps['preprocessor'].transform(pd.DataFrame.from_records(data))

array([[   0,    1,    0,    1,    0,    0,    1,    0,    0,    0,    0,
           0,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    1,    0,   67, 5076,    6]], dtype=int64)

In [76]:
clf.named_steps['preprocessor'].transform(df_X.loc[:0])

array([[   0,    1,    0,    1,    0,    0,    1,    0,    0,    0,    0,
           0,    0,    1,    1,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    1,    0,   67, 5076,    6]], dtype=int64)

In [100]:
clf.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].feature_names_in_ #.named_features['onehot']

array(['Geschlecht', 'Arbeitsstelle', 'Wohnsituation', 'Sparkonto',
       'Lohnkonto', 'Verwendungszweck'], dtype=object)

In [101]:
clf.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].categories_

[array(['männlich', 'weiblich'], dtype=object),
 array(['hohe Qualifikation', 'mittlere Qualifikation',
        'niedrige Qualifikation - permanent',
        'niedrige Qualifikation - temporär'], dtype=object),
 array(['Eigentum', 'Miete', 'kostenlos'], dtype=object),
 array(['gering', 'hoch', 'mittel', 'sehr hoch', nan], dtype=object),
 array(['gering', 'hoch', 'mittel', nan], dtype=object),
 array(['Ausbildung', 'Fahrzeug', 'Ferien/andere', 'Geschäft',
        'Haushaltsgeräte', 'Unterhalt', 'Unterhaltungselektronik',
        'Wohnungsreinrichtung'], dtype=object)]

In [114]:
cat_features_cats = [(feat, list(clf.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].categories_[i])) for i, feat in enumerate(cat_features)]

In [115]:
cat_features_cats

[('Geschlecht', ['männlich', 'weiblich']),
 ('Arbeitsstelle',
  ['hohe Qualifikation',
   'mittlere Qualifikation',
   'niedrige Qualifikation - permanent',
   'niedrige Qualifikation - temporär']),
 ('Wohnsituation', ['Eigentum', 'Miete', 'kostenlos']),
 ('Sparkonto', ['gering', 'hoch', 'mittel', 'sehr hoch', nan]),
 ('Lohnkonto', ['gering', 'hoch', 'mittel', nan]),
 ('Verwendungszweck',
  ['Ausbildung',
   'Fahrzeug',
   'Ferien/andere',
   'Geschäft',
   'Haushaltsgeräte',
   'Unterhalt',
   'Unterhaltungselektronik',
   'Wohnungsreinrichtung'])]

In [109]:
num_features

['Alter', 'Kreditbetrag', 'Laufzeit']

In [102]:
clf.predict(pd.DataFrame.from_records(data))

array([1], dtype=int64)

In [104]:
dump(clf, os.path.join('models', 'clf.joblib'))

['models\\clf.joblib']

In [105]:
l = load(os.path.join('models', 'clf.joblib'))